In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd
import torch.utils.data
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import warnings
import os

warnings.filterwarnings("ignore")

In [ ]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import torchvision

custom_class_mapping = {
    'CT': 0,
    'PN': 1,
    'MP': 2,
    'NC': 3,
    'IC': 4,
    'WM': 5,
}

transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ToTensor(),
])

test_dataset = torchvision.datasets.ImageFolder(root='test_data', transform=transform)
test_dataset.class_to_idx = custom_class_mapping
print("Class to Index Mapping:", test_dataset.class_to_idx)
test_dataset.idx_to_class = {v: k for k, v in test_dataset.class_to_idx.items()}
print("Index to Class Mapping:", test_dataset.idx_to_class)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)

Class to Index Mapping: {'CT': 0, 'PN': 1, 'MP': 2, 'NC': 3, 'IC': 4, 'WM': 5}
Index to Class Mapping: {0: 'CT', 1: 'PN', 2: 'MP', 3: 'NC', 4: 'IC', 5: 'WM'}


In [ ]:
class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max = torch.max(x, 1)[0].unsqueeze(1)
        avg = torch.mean(x, 1).unsqueeze(1)
        concat = torch.cat((max, avg), dim=1)
        output = self.conv(concat)
        output = output * x
        return output

class CAM(nn.Module):
    def __init__(self, channels, r):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=self.channels, out_features=self.channels // self.r, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=self.channels // self.r, out_features=self.channels, bias=True)
        )

    def forward(self, x):
        max = F.adaptive_max_pool2d(x, output_size=1)
        avg = F.adaptive_avg_pool2d(x, output_size=1)
        b, c, _, _ = x.size()
        linear_max = self.linear(max.view(b, c)).view(b, c, 1, 1)
        linear_avg = self.linear(avg.view(b, c)).view(b, c, 1, 1)
        output = linear_max + linear_avg
        output = torch.sigmoid(output) * x
        return output

class CBAM(nn.Module):
    def __init__(self, channels, r):
        super(CBAM, self).__init__()
        self.channels = channels
        self.r = r
        self.sam = SAM(bias=False)
        self.cam = CAM(channels=self.channels, r=self.r)

    def forward(self, x):
        output = self.cam(x)
        output = self.sam(output)
        return output + x

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
    )

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.decoder(x)


class VGG19_CBAM(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGG19_CBAM, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.encoder = Encoder(in_channels=self.in_channels, out_channels=64)

        self.conv_block1 = nn.Sequential(
            conv_block(64, 64),
            CBAM(64, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block2 = nn.Sequential(
            conv_block(64, 128),
            CBAM(128, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block3 = nn.Sequential(
            conv_block(128, 256),
            *[conv_block(256, 256) for _ in range(2)],
            CBAM(256, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block4 = nn.Sequential(
            conv_block(256, 512),
            *[conv_block(512, 512) for _ in range(2)],
            CBAM(512, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block5 = nn.Sequential(
            *[conv_block(512, 512) for _ in range(3)],
            CBAM(512, r=4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder1 = Decoder(in_channels=512, out_channels=256)
        self.decoder2 = Decoder(in_channels=256, out_channels=128)
        self.decoder3 = Decoder(in_channels=128, out_channels=64)
        self.decoder4 = Decoder(in_channels=64, out_channels=in_channels)

        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))

        self.linear1 = nn.Sequential(
            nn.Linear(in_features=3 * 7 * 7, out_features=4096, bias=True),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.linear2 = nn.Sequential(
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.linear3 = nn.Linear(in_features=4096, out_features=self.out_channels, bias=True)

    def forward(self, x):
        x = self.encoder(x)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)

        x = self.decoder1(x)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)

        x = self.avg_pool(x)

        x = x.view(x.shape[0], -1)

        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x


In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, ce_weight=0.5, focal_weight=0.3, mcc_weight=0.1, f1_weight=0.1, epsilon=1e-7, label_smoothing=0.1):
        super(CustomLoss, self).__init__()
        self.gamma = gamma
        self.ce_weight = ce_weight
        self.focal_weight = focal_weight
        self.mcc_weight = mcc_weight
        self.f1_weight = f1_weight
        self.epsilon = epsilon
        self.label_smoothing = label_smoothing

        if alpha is None:
            self.alpha = torch.ones(6)
        else:
            self.alpha = torch.tensor(alpha, dtype=torch.float32)

    def forward(self, y_pred, labels, weights=None):
        y_true = F.one_hot(labels, num_classes=y_pred.size(1)).float()
        y_true = y_true * (1 - self.label_smoothing) + self.label_smoothing / y_pred.size(1)

        if self.alpha is not None:
            self.alpha = self.alpha.to(y_pred.device)

        y_pred = torch.clamp(F.softmax(y_pred, dim=1), self.epsilon, 1.0 - self.epsilon)

        ce_loss = -torch.sum(y_true * torch.log(y_pred), dim=1)

        focal_loss = -torch.sum(self.alpha * (1 - y_pred) ** self.gamma * y_true * torch.log(y_pred), dim=1)

        if weights is None:
            weights = torch.ones(y_true.shape[0], dtype=torch.float32).to(y_pred.device)
        else:
            weights = weights.to(y_pred.device)

        tp = torch.sum(weights[:, None] * y_true * y_pred, dim=0)
        tn = torch.sum(weights[:, None] * (1 - y_true) * (1 - y_pred), dim=0)
        fp = torch.sum(weights[:, None] * (1 - y_true) * y_pred, dim=0)
        fn = torch.sum(weights[:, None] * y_true * (1 - y_pred), dim=0)

        denominator = torch.sqrt((tp + fp + self.epsilon) * (tp + fn + self.epsilon) * (tn + fp + self.epsilon) * (tn + fn + self.epsilon))

        numerator = tp * tn - fp * fn
        mcc_loss = 1.0 - torch.sum(numerator / (denominator + self.epsilon))

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)
        f1_score = 2 * (precision * recall) / (precision + recall + self.epsilon)

        f1_loss = 1.0 - f1_score.mean()

        total_loss = (self.ce_weight * ce_loss.mean() +
                      self.focal_weight * focal_loss.mean() +
                      self.mcc_weight * mcc_loss +
                      self.f1_weight * f1_loss)

        return total_loss


In [ ]:
class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max_pool = torch.max(x, 1)[0].unsqueeze(1)
        avg_pool = torch.mean(x, 1).unsqueeze(1)
        concat = torch.cat((max_pool, avg_pool), dim=1)
        output = self.conv(concat)
        output = torch.sigmoid(output)  # Apply sigmoid to get attention map
        return output * x

class CAM(nn.Module):
    def __init__(self, channels, r=16):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=channels, out_features=channels // r, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=channels // r, out_features=channels, bias=True)
        )

    def forward(self, x):
        max_pool = F.adaptive_max_pool2d(x, output_size=1)
        avg_pool = F.adaptive_avg_pool2d(x, output_size=1)
        max_pool = max_pool.view(max_pool.size(0), -1)
        avg_pool = avg_pool.view(avg_pool.size(0), -1)
        max_out = self.linear(max_pool)
        avg_out = self.linear(avg_pool)
        out = torch.sigmoid(max_out + avg_out).view(x.size(0), x.size(1), 1, 1)
        return out * x

class CBAM(nn.Module):
    def __init__(self, channels, r=16):
        super(CBAM, self).__init__()
        self.cam = CAM(channels, r)
        self.sam = SAM()

    def forward(self, x):
        out = self.cam(x)
        out = self.sam(out)
        return out

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
            out_channels, out_channels * self.expansion, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.cbam = CBAM(out_channels * self.expansion)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        out = self.cbam(out)

        return out

In [ ]:
class GAFNET_CBAM(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(GAFNET_CBAM, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(Bottleneck, 64, 3)
        self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(Bottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 512, 3, stride=2)
        self.layer5 = self._make_layer(Bottleneck, 1024, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024 * Bottleneck.expansion, num_classes)

        self.skip_connection = nn.Conv2d(
            in_channels,
            1024 * Bottleneck.expansion,
            kernel_size=1,
            stride=32,
            bias=False,
        )
        self.skip_pool = nn.AdaptiveAvgPool2d((1, 1))

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels * block.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        skip = self.skip_connection(x)
        skip = self.skip_pool(skip)  # Ensure the skip connection is resized properly
        skip = torch.flatten(skip, 1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        # Ensure the dimensions match before adding
        if x.shape[1] == skip.shape[1]:
            x += skip

        return x


In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, ce_weight=0.5, focal_weight=0.3, mcc_weight=0.1, f1_weight=0.1, epsilon=1e-7, label_smoothing=0.1):
        super(CustomLoss, self).__init__()
        self.gamma = gamma
        self.ce_weight = ce_weight
        self.focal_weight = focal_weight
        self.mcc_weight = mcc_weight
        self.f1_weight = f1_weight
        self.epsilon = epsilon
        self.label_smoothing = label_smoothing

        if alpha is None:
            self.alpha = torch.ones(6)
        else:
            self.alpha = torch.tensor(alpha, dtype=torch.float32)

    def forward(self, y_pred, labels, weights=None):
        y_true = F.one_hot(labels, num_classes=y_pred.size(1)).float()

        y_true = y_true * (1 - self.label_smoothing) + self.label_smoothing / y_pred.size(1)

        if self.alpha is not None:
            self.alpha = self.alpha.to(y_pred.device)

        y_pred = torch.clamp(F.softmax(y_pred, dim=1), self.epsilon, 1.0 - self.epsilon)

        ce_loss = -torch.sum(y_true * torch.log(y_pred), dim=1)

        focal_loss = -torch.sum(self.alpha * (1 - y_pred) ** self.gamma * y_true * torch.log(y_pred), dim=1)

        if weights is None:
            weights = torch.ones(y_true.shape[0], dtype=torch.float32).to(y_pred.device)
        else:
            weights = weights.to(y_pred.device)

        tp = torch.sum(weights[:, None] * y_true * y_pred, dim=0)
        tn = torch.sum(weights[:, None] * (1 - y_true) * (1 - y_pred), dim=0)
        fp = torch.sum(weights[:, None] * (1 - y_true) * y_pred, dim=0)
        fn = torch.sum(weights[:, None] * y_true * (1 - y_pred), dim=0)

        denominator = torch.sqrt((tp + fp + self.epsilon) * (tp + fn + self.epsilon) * (tn + fp + self.epsilon) * (tn + fn + self.epsilon))

        numerator = tp * tn - fp * fn
        mcc_loss = 1.0 - torch.sum(numerator / (denominator + self.epsilon))

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)
        f1_score = 2 * (precision * recall) / (precision + recall + self.epsilon)

        f1_loss = 1.0 - f1_score.mean()

        total_loss = (self.ce_weight * ce_loss.mean() +
                      self.focal_weight * focal_loss.mean() +
                      self.mcc_weight * mcc_loss +
                      self.f1_weight * f1_loss)

        return total_loss


In [ ]:
class SAM(nn.Module):
    def __init__(self, bias=False):
        super(SAM, self).__init__()
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=7, stride=1, padding=3, dilation=1, bias=self.bias)

    def forward(self, x):
        max = torch.max(x,1)[0].unsqueeze(1)
        avg = torch.mean(x,1).unsqueeze(1)
        concat = torch.cat((max,avg), dim=1)
        output = self.conv(concat)
        output = output * x
        return output

class CAM(nn.Module):
    def __init__(self, channels, r):
        super(CAM, self).__init__()
        self.channels = channels
        self.r = r
        self.linear = nn.Sequential(
            nn.Linear(in_features=self.channels, out_features=self.channels//self.r, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=self.channels//self.r, out_features=self.channels, bias=True))

    def forward(self, x):
        max = F.adaptive_max_pool2d(x, output_size=1)
        avg = F.adaptive_avg_pool2d(x, output_size=1)
        b, c, _, _ = x.size()
        linear_max = self.linear(max.view(b,c)).view(b, c, 1, 1)
        linear_avg = self.linear(avg.view(b,c)).view(b, c, 1, 1)
        output = linear_max + linear_avg
        output = F.sigmoid(output) * x
        return output

class CBAM(nn.Module):
    def __init__(self, channels, r=16):
        super(CBAM, self).__init__()
        self.channels = channels
        self.r = r
        self.sam = SAM(bias=False)
        self.cam = CAM(channels=self.channels, r=self.r)

    def forward(self, x):
        output = self.cam(x)
        output = self.sam(output)
        return output + x

def conv_block(in_channels, out_channels):
    return nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                         nn.BatchNorm2d(out_channels),
                         nn.ReLU())

class VGG19_CBAM2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv_block1 = nn.Sequential(nn.Conv2d(in_channels=self.in_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU(),
                                         conv_block(64, 64),
                                         CBAM(64, r=4),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block2 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU(),
                                         conv_block(128, 128),
                                         CBAM(128, r=4),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block3 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU(),
                                         *[conv_block(256, 256) for _ in range(3)],
                                         CBAM(256, r=4),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block4 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU(),
                                         *[conv_block(512, 512) for _ in range(3)],
                                         CBAM(512, r=4),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block5 = nn.Sequential(*[conv_block(512, 512) for _ in range(4)],
                                         CBAM(512, r=4),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(7,7))

        self.linear1 = nn.Sequential(nn.Linear(in_features=7*7*512, out_features=4096, bias=True),
                                     nn.Dropout(0.5),
                                     nn.ReLU())
        self.linear2 = nn.Sequential(nn.Linear(in_features=4096, out_features=4096, bias=True),
                                     nn.Dropout(0.5),
                                     nn.ReLU())
        self.linear3 = nn.Linear(in_features=4096, out_features=self.out_channels, bias=True)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = self.avg_pool(x)
        x = self.linear1(x.view(x.shape[0], -1))
        x = self.linear2(x)
        x = self.linear3(x)
        return x

In [ ]:
model3 = VGG19_CBAM2(in_channels=3, out_channels=6)
state_dict = torch.load("saved_models/model3.pth")
model3.load_state_dict(state_dict)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.005)
criterion = nn.CrossEntropyLoss()
total_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model3 = model3.to(device)


model2 = GAFNET_CBAM(in_channels=3,num_classes=6)
state_dict = torch.load("saved_models/model2.pth")
model2.load_state_dict(state_dict)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.005)
criterion = CustomLoss()
total_epochs = 8
model2 = model2.to(device)

model = VGG19_CBAM(in_channels=3, out_channels=6)
state_dict = torch.load("saved_models/model1.pth")
model.load_state_dict(state_dict)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.005)
criterion = CustomLoss()
total_epochs = 10
model = model.to(device)

model.eval()
model2.eval()
model3.eval()

In [ ]:
w1, w2, w3 = 0.546, 0.464, 0.527
total_weight = w1 + w2 + w3

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

predictions = []
true_labels = []

for batch_idx, (images, labels) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        output1 = model(images)
        output2 = model2(images)
        output3 = model3(images)

        weighted_avg_outputs = (w1 * output1 + w2 * output2 + w3 * output3) / total_weight
        _, preds = torch.max(weighted_avg_outputs, 1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

    if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(test_loader):
        print(f"Processed {batch_idx + 1}/{len(test_loader)} batches.")

predictions = np.array(predictions)
true_labels = np.array(true_labels)

accuracy = accuracy_score(true_labels, predictions)

f1 = f1_score(true_labels, predictions, average='macro')

precision = precision_score(true_labels, predictions, average='macro')

sensitivity = recall_score(true_labels, predictions, average='macro')

conf_matrix = confusion_matrix(true_labels, predictions)
specificity_per_class = []
for i in range(len(custom_class_mapping)):
    tn = conf_matrix.sum() - conf_matrix[i, :].sum() - conf_matrix[:, i].sum() + conf_matrix[i, i]
    fp = conf_matrix[:, i].sum() - conf_matrix[i, i]
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_per_class.append(specificity)
specificity = np.mean(specificity_per_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")


Processed 100/1125 batches.
Processed 200/1125 batches.
Processed 300/1125 batches.
Processed 400/1125 batches.
Processed 500/1125 batches.
Processed 600/1125 batches.
Processed 700/1125 batches.
Processed 800/1125 batches.
Processed 900/1125 batches.
Processed 1000/1125 batches.
Processed 1100/1125 batches.
Processed 1125/1125 batches.
Accuracy: 0.9673
F1-Score: 0.9634
Precision: 0.9670
Sensitivity (Recall): 0.9600
Specificity: 0.9860
